In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
import cv2
from datetime import timedelta
import math
import os
import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import PIL.Image
from IPython.display import display
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


/home/vishal/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path = "/home/vishal/dogbreed/train"
labels=pd.read_csv("/home/vishal/dogbreed/labels.csv")
train_dir = os.listdir( path )

x_train=[]              #loading data and labels
y_train=[]

for img_name in tqdm(train_dir):
    y_train.append((labels[labels["id"]==img_name.split(".")[0]]["breed"].values[0]))  #Setting labels id = iamge name
    x_train.append(cv2.resize((cv2.imread(path+"/"+img_name)),(97,97)).reshape(1,97,97,3)) #Reading and resizing the image

100%|██████████| 10222/10222 [00:26<00:00, 386.50it/s]


In [3]:
y_train=pd.get_dummies(y_train).values  #Doing one hot encoding
x_train=np.concatenate(x_train,axis=0)  #Stacking 10222 images in numpy array

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, test_size=0.2, random_state=4) #Splitting data into training and test sets

In [6]:
def batch(x,y,start,batch_size):         #Defining a batch function for sending the images an labels array in batches 
    x_batch=x[start:start+batch_size]
    y_batch=y[start:start+batch_size]
    start=start+batch_size
    return x_batch,y_batch,start

In [7]:
lr=.001
batch_size=250
n_epochs=3

In [8]:
x=tf.placeholder(tf.float32,[None,97,97,3],name="image")   #Creating placeholders for x&y as well as initializing weights and biases
x_image = tf.reshape(x, [-1,97,97,3])                      #Reshaping image size for conv1 layer input
y=tf.placeholder(tf.float32,[None,120],name="labels")
w=tf.Variable(tf.random_normal(shape=[97*97*3,120]),name="weights")
b=tf.Variable(tf.zeros([1,120]),name="bias")

In [9]:
#CNN MODEL:
conv1=tf.layers.conv2d(inputs=x_image,filters=32,kernel_size=[5,5],padding="SAME",activation=tf.nn.relu)
pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=[3, 3], strides=2)

conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],padding="SAME",activation=tf.nn.relu)
pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

pool2_flat=tf.reshape(pool2,[-1,24*24*64])
dense=tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

keep_prob=tf.placeholder(tf.float32)
drop_layer=tf.nn.dropout(dense,keep_prob)

logits = tf.layers.dense(inputs=drop_layer, units=120)
entropy=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y)
loss=tf.reduce_mean(entropy)
optimizer=tf.train.GradientDescentOptimizer(lr).minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
test_size =8
num_steps=32
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    
    for epoch in tqdm(range(n_epochs)):
        start=0
        for step in range(num_steps):
            x_batch,y_batch,start= batch(x_train,y_train,start,batch_size)
            if step % 10 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                x:x_batch, y: y_batch, keep_prob: 1.0})
            
            optimizer.run(feed_dict={x: x_batch, y: y_batch, keep_prob: 0.5})
        print('Epoch %d, step %d, training accuracy %f' %(epoch,step, train_accuracy))

    test_accuracy = 0.0
    start=0
    for i in xrange(test_size):
        x_batch,y_batch,start= batch(x_test,y_test,start,batch_size)
        acc = accuracy.eval(feed_dict={x: x_batch, y:y_batch, keep_prob: 1.0})
        if i % 10 == 0:
            print('%d: test accuracy %f' % (i, acc))
            test_accuracy += acc
    print 'avg test accuracy:', 100*test_accuracy/(test_size)

 33%|███▎      | 1/3 [02:24<04:48, 144.12s/it]

Epoch 0, step 31, training accuracy 0.012000


 67%|██████▋   | 2/3 [04:59<02:29, 149.63s/it]

Epoch 1, step 31, training accuracy 0.012000


100%|██████████| 3/3 [07:30<00:00, 150.18s/it]

Epoch 2, step 31, training accuracy 0.008000


0: test accuracy 0.008000
avg test accuracy: 0.10000000474974513
